In [ ]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --verify-certificate False --cluster-id j-MJ811CE3VFGB --auth-type None

In [ ]:
import pyspark
from functools import reduce
from operator import add
from pytz import timezone
from pyspark import  StorageLevel, SparkContext, SparkConf
import pyspark.sql.types  as st
import pyspark.sql.functions  as func
from pyspark.sql.functions import *
from pyspark.sql.window import Window as W
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from datetime import datetime, timedelta
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime, timedelta


conf = SparkConf()
spark_cfgs = {
    "spark.app.name": "MMIX - Weekly ETL",
    "spark.hadoop.hive.exec.dynamic.partition.mode": "nonstrict",
    "spark.spark.sql.legacy.timeParserPolicy": "LEGACY",
    "spark.sql.parquet.compression.codec": "SNAPPY",
    "spark.sql.sources.partitionOverwriteMode": "dynamic",
}

for k, v in spark_cfgs.items():
    conf.set(k, v)
spark = (SparkSession.builder.enableHiveSupport().config(conf=conf).getOrCreate()
)
spark

In [ ]:
table_paths = [
    ('mmix_net_sales_weekly', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_net_sales_weekly/'),
    ('mmix_sales_weekly', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_sales_weekly/'),
    ('mmix_speaker_weekly', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_speaker_weekly/'),
    ('mmix_tam_ce_weekly', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_tam_ce_weekly/'),
    ('mmix_tam_dtc_weekly', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_tam_dtc_weekly/'), 
    ('mmix_product_sales_weekly', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_product_sales_weekly/'), 
    ('mmix_xo_weekly_staging', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_xo_weekly_staging/'), 
    
]

for table_name, input_path in table_paths:
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.{table_name}
    USING PARQUET
    LOCATION '{input_path}'
    """)
    print(f"Table {table_name}_new created")

print("All tables complete")


In [ ]:
tam_combined = spark.sql(
    """
        select 
            year_week, 
            product_brand_name, 
            zip, 
            audience,
            channel,
            engage,
            reach, 
            cost 
        from 
            sandbox_measurement_analyst.mmix_tam_ce_weekly 
        union all 
        select 
            year_week, 
            product_brand_name, 
            zip, 
            audience,
            channel,
            engage,
            reach, 
            cost 
        from 
            sandbox_measurement_analyst.mmix_tam_dtc_weekly 
    """
)
tam_combined.registerTempTable("tam_combined")

In [ ]:
df_prepivot = spark.sql(
    """
    with combined_data as (
        select distinct
            product_brand_name,
            year_week,
            zip as zip_code,
            audience,
            channel,
            reach,
            engage,
            cost
        from
            sandbox_measurement_analyst.mmix_xo_weekly_staging
        union all
        select distinct
            product_brand_name,
            year_week,
            zip as zip_code,
            audience,
            channel,
            reach,
            engage,
            cost
        from
            tam_combined
        union all
        select distinct
            product_brand_name,
            year_week,
            zip as zip_code,
            audience,
            channel,
            reach,
            engage,
            cost
        from
            sandbox_measurement_analyst.mmix_speaker_weekly
    )

    select * from combined_data
    """
)
df_prepivot.createOrReplaceTempView("combined")

In [ ]:
df_pivoted = (
    spark.sql(
        """
SELECT
    product_brand_name,
    zip_code,
    year_week,
    channel || '_' || audience AS channel_audience,
    reach,
    engage,
    cost
FROM combined
"""
    )
    .groupBy(
        [
            "product_brand_name",
            "zip_code",
            "year_week"
        ]
    )
    .pivot("channel_audience")
    .sum("reach", "engage", "cost")
)

df_pivoted = df_pivoted.select(
    *[
        F.col(col).alias(col.replace("_sum(", "_").replace(")", ""))
        for col in df_pivoted.columns
    ]
)
df_pivoted.printSchema()
df_pivoted.createOrReplaceTempView("post_pivot")

In [ ]:
df_final = spark.sql(
"""
    with cte as (
        select distinct
            coalesce(
                pivoted.product_brand_name,
                sales.product_brand_name,
                hcp_count.product_brand_name
            ) as product_brand_name,
            coalesce(pivoted.zip_code, sales.zip, hcp_count.zip_code) as zip_code,
            coalesce(pivoted.year_week, sales.year_week) as date_year_week,
            addressable_tv_dtc_cost,
            addressable_tv_dtc_engage,
            addressable_tv_dtc_reach,
            alert_dtc_cost,
            alert_dtc_engage,
            alert_dtc_reach,
            alert_hcp_cost,
            alert_hcp_engage,
            alert_hcp_reach,
            connected_tv_dtc_cost,
            connected_tv_dtc_engage,
            connected_tv_dtc_reach,
            display_dtc_cost,
            display_dtc_engage,
            display_dtc_reach,
            display_hcp_cost,
            display_hcp_engage,
            display_hcp_reach,
            dr_cable_dtc_cost,
            dr_cable_dtc_engage,
            dr_cable_dtc_reach,
            e_detail_hcp_cost,
            e_detail_hcp_engage,
            e_detail_hcp_reach,
            ehr_emr_hcp_cost,
            ehr_emr_hcp_engage,
            ehr_emr_hcp_reach,
            enewsletter_dtc_cost,
            enewsletter_dtc_engage,
            enewsletter_dtc_reach,
            enewsletter_hcp_cost,
            enewsletter_hcp_engage,
            enewsletter_hcp_reach,
            hispanic_connected_tv_dtc_cost,
            hispanic_connected_tv_dtc_engage,
            hispanic_connected_tv_dtc_reach,
            hispanic_linear_tv_dtc_cost,
            hispanic_linear_tv_dtc_engage,
            hispanic_linear_tv_dtc_reach,
            house_email_dtc_cost,
            house_email_dtc_engage,
            house_email_dtc_reach,
            house_email_hcp_cost,
            house_email_hcp_engage,
            house_email_hcp_reach,
            journals_print_dtc_cost,
            journals_print_dtc_engage,
            journals_print_dtc_reach,
            journals_print_hcp_cost,
            journals_print_hcp_engage,
            journals_print_hcp_reach,
            linear_tv_dtc_cost,
            linear_tv_dtc_engage,
            linear_tv_dtc_reach,
            monograph_hcp_cost,
            monograph_hcp_engage,
            monograph_hcp_reach,
            online_video_dtc_cost,
            online_video_dtc_engage,
            online_video_dtc_reach,
            paid_search_dtc_cost,
            paid_search_dtc_engage,
            paid_search_dtc_reach,
            paid_search_hcp_cost,
            paid_search_hcp_engage,
            paid_search_hcp_reach,
            paid_social_dtc_cost,
            paid_social_dtc_engage,
            paid_social_dtc_reach,
            paid_social_hcp_cost,
            paid_social_hcp_engage,
            paid_social_hcp_reach,
            point_of_care_dtc_cost,
            point_of_care_dtc_engage,
            point_of_care_dtc_reach,
            puerto_rico_local_tv_dtc_cost,
            puerto_rico_local_tv_dtc_engage,
            puerto_rico_local_tv_dtc_reach,
            speaker_program_dtc_cost,
            speaker_program_dtc_engage,
            speaker_program_dtc_reach,
            speaker_program_hcp_cost,
            speaker_program_hcp_engage,
            speaker_program_hcp_reach,
            streaming_audio_dtc_cost,
            streaming_audio_dtc_engage,
            streaming_audio_dtc_reach,
            streaming_tv_dtc_cost,
            streaming_tv_dtc_engage,
            streaming_tv_dtc_reach,
            tam_dtc_cost,
            tam_dtc_engage,
            tam_dtc_reach,
            tam_hd_ce_cost,
            tam_hd_ce_engage,
            tam_hd_ce_reach,
            third_party_email_dtc_cost,
            third_party_email_dtc_engage,
            third_party_email_dtc_reach,
            third_party_email_hcp_cost,
            third_party_email_hcp_engage,
            third_party_email_hcp_reach,
            third_party_site_content_hcp_cost,
            third_party_site_content_hcp_engage,
            third_party_site_content_hcp_reach,
            video_dtc_cost,
            video_dtc_engage,
            video_dtc_reach,
            video_hcp_cost,
            video_hcp_engage,
            video_hcp_reach,
            sales.sales_qty as gne_867_sales_qty_eqv,
            hcp_count.hcp_count
        from post_pivot as pivoted
        full outer join
            sandbox_measurement_analyst.mmix_sales_weekly as sales
            on
                pivoted.product_brand_name = sales.product_brand_name
                and pivoted.year_week = sales.year_week
                and pivoted.zip_code = sales.zip
        full outer join
            sandbox_measurement_analyst.mars_hcp_count_staging as hcp_count
            on
                pivoted.product_brand_name = hcp_count.product_brand_name
                and pivoted.zip_code = hcp_count.zip_code
    )
    select * from cte 
    where date_year_week is not null
""")

In [ ]:
output_path = f's3://cmg-oasis-prod-sandbox-measurement-analyst/mmix_weekly_outputs/mmix_pivoted/'
today = datetime.today().strftime('%Y%m%d')
output_database = 'sandbox_measurement_analyst'  
table_name = 'mars_combined_pivoted_weekly'


df_final.write.mode("overwrite").parquet(
    f"{output_path}{today}/"
)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {output_database}.{table_name}
    USING PARQUET
    LOCATION '{output_path}{today}/'
""")

In [ ]:
df = spark.sql("""select * from sandbox_measurement_analyst.mars_combined_pivoted_weekly""")
df.write \
    .partitionBy("product_brand_name", "date_year_week") \
    .parquet("s3://cmg-oasis-prod-sandbox-measurement-analyst/mars_pivoted_weekly_partitioned_20240719/")